In [3]:
import pandas as pd   # pip install pandas

df_faces = pd.read_excel('../stats/Преступность/11/ЛИЦА.xls', sheet_name='ЛИЦА-1', index_col=0) #считываем excel файл
#df_faces = df_faces.reset_index() #
df_faces = df_faces.rename(columns=lambda x: x if not 'Unnamed' in str(x) else None)
df_faces = df_faces.dropna()
df_faces.columns = pd.Series(df_faces.columns).fillna(method='ffill', axis=0)

In [4]:
def read_excel(data, path, sheet):
    data = pd.read_excel(path, sheet_name=sheet, index_col=0) #считываем excel файл
    data = data.reset_index() #
    data = data.rename(columns=lambda x: x if not 'Unnamed' in str(x) else None)
    #data = data.dropna()
    data.columns = pd.Series(data.columns).fillna(method='ffill', axis=0)
    return data

df_stats_all = pd.DataFrame()
df_stats_all = read_excel(df_stats_all, 'outputMain.xlsx', 'all')
df_stats_all_c = df_stats_all.copy()

In [5]:
df_faces.index = df_faces.index.str.lower()

In [6]:
df_faces_c = df_faces.copy()

In [7]:
df_faces_c = df_faces_c.loc[['год', 'итого'], 'ВСЕГО']
df_faces_c = df_faces_c.T[df_faces_c.T['год'] > 2013]
df_faces_c = df_faces_c[df_faces_c['год'] < 2019]
df_faces_c = df_faces_c.T
df_faces_c

,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО,ВСЕГО
"СВЕДЕНИЯ О ЛИЦАХ, СОВЕРШИВШИХ ПРЕСТУПЛЕНИЯ",,,,,
год,2014.0,2015.0,2016.0,2017.0,2018.0
итого,24763.0,26665.0,26039.0,23465.0,22786.0


In [8]:
df_faces_c.columns = df_faces_c.iloc[0]
df_faces_c = df_faces_c.drop(df_faces_c.index[0])

In [9]:
df_faces_c.T

"СВЕДЕНИЯ О ЛИЦАХ, СОВЕРШИВШИХ ПРЕСТУПЛЕНИЯ",итого
год,
2014.0,24763.0
2015.0,26665.0
2016.0,26039.0
2017.0,23465.0
2018.0,22786.0


In [10]:
df_stats_all_c.loc[[6]]

,index,2014,2015,2016,2017,2018
6,безработные – всего,83.4,94.8,99.6,78.2,70.3


In [11]:
df_stats_all_c = df_stats_all.copy()
df_stats_all_c = df_stats_all_c.loc[[6]]
df_stats_all_c = df_stats_all_c.T
df_stats_all_c = df_stats_all_c.drop(df_stats_all_c.index[0])
df_stats_all_c

,6
2014,83.4
2015,94.8
2016,99.6
2017,78.2
2018,70.3


Проверка на нормальность

In [12]:
from scipy import stats

[stats.shapiro(df_faces_c.squeeze()), stats.shapiro(df_stats_all_c.squeeze())]

[ShapiroResult(statistic=0.9445344805717468, pvalue=0.698148250579834),
 ShapiroResult(statistic=0.9609274864196777, pvalue=0.8144269585609436)]

Данные имеют нормальное распределение. Можно рассчитвать корреляцию Пирсона

In [13]:
stats.pearsonr(df_faces_c.squeeze(), df_stats_all_c.squeeze())

(0.9534474427541751, 0.011972727440374102)

Наблюдаем очень высокий коэффициент корреляции между значением преступности и уровнем безработицы в алтайском крае

In [14]:
df_stud = read_excel(df_stats_all, 'outputStudMain.xlsx', 'Sheet1')
df_stud_c = df_stud.copy()
df_stud_c

,index,2014,2015,2016,2017,2018
0,обучающиеся,245057,251348,259214,265580,269885
1,чел СПО,31659,33161,47310,47533,48342
2,принято СПО,10384,10758,15047,15855,16282
3,выпущено СПО,6950,6620,11734,11899,11585
4,чел высш,69252,60711,52127,51129,50297
5,принято чел высш,14520,14080,12984,13780,13197
6,выпущено чел высш,14836,15092,13265,10458,10255


In [15]:
df_stud_c = df_stud_c.T
df_stud_c = df_stud_c.drop(df_stud_c.index[0])
df_stud_c

,0,1,2,3,4,5,6
2014,245057,31659,10384,6950,69252,14520,14836
2015,251348,33161,10758,6620,60711,14080,15092
2016,259214,47310,15047,11734,52127,12984,13265
2017,265580,47533,15855,11899,51129,13780,10458
2018,269885,48342,16282,11585,50297,13197,10255


Проверка на нормальность

In [16]:
[stats.shapiro(df_stud_c[0]), stats.shapiro(df_stud_c[1]), stats.shapiro(df_stud_c[2]), stats.shapiro(df_stud_c[3]),stats.shapiro(df_stud_c[4]),stats.shapiro(df_stud_c[5]),stats.shapiro(df_stud_c[6])] 

[ShapiroResult(statistic=0.9653968811035156, pvalue=0.8449691534042358),
 ShapiroResult(statistic=0.7479941844940186, pvalue=0.028514130041003227),
 ShapiroResult(statistic=0.8094250559806824, pvalue=0.09649751335382462),
 ShapiroResult(statistic=0.7357102632522583, pvalue=0.02182762511074543),
 ShapiroResult(statistic=0.8340088129043579, pvalue=0.1490303874015808),
 ShapiroResult(statistic=0.9588518738746643, pvalue=0.7999665141105652),
 ShapiroResult(statistic=0.8457009196281433, pvalue=0.18130750954151154)]

Видим, что у датафремов df_stud_c с индексами 1 и 3 распределение отлично от норамльного, т.к. pvalue < 0.05. Для них будем расчитывать корреляцию Кендалла

In [17]:
df_stud_c[0]


2014    245057
2015    251348
2016    259214
2017    265580
2018    269885
Name: 0, dtype: object

In [18]:
stats.rankdata(df_stats_all_c.squeeze())

array([3., 4., 5., 2., 1.])

Корреляция между уровнем безработицы и кол-вом обучающихся всего

In [19]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[0])


(-0.5211450195316683, 0.36785395777477875)

Корреляция между уровнем безработицы и кол-вом человек в учреждениях СПО

In [20]:
stats.kendalltau(stats.rankdata(df_stats_all_c.squeeze()), stats.rankdata(df_stud_c[1]))


KendalltauResult(correlation=-0.39999999999999997, pvalue=0.48333333333333334)

Корреляция между уровнем безработицы и кол-вом принятых человек в учреждения СПО

In [21]:
stats.spearmanr(df_stats_all_c.squeeze(), df_stud_c[2])


SpearmanrResult(correlation=-0.6, pvalue=0.28475697986529375)

Корреляция между уровнем безработицы и кол-вом выпустников из учреждений СПО

In [22]:
stats.kendalltau(stats.rankdata(df_stats_all_c.squeeze()), stats.rankdata(df_stud_c[3]))


KendalltauResult(correlation=-0.19999999999999998, pvalue=0.8166666666666667)

Корреляция между уровнем безработицы и кол-вом человек в ВУЗах

In [23]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[4])


(0.21596180842636714, 0.7271815128037152)

Корреляция между уровнем безработицы и кол-вом человек принятых в ВУЗ

In [24]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[5])


(-0.039997505830881186, 0.9490871758271164)

Корреляция между уровнем безработицы и кол-вом выпустников из ВУЗов

In [25]:
stats.pearsonr(df_stats_all_c.squeeze(), df_stud_c[6])

(0.7132375534801394, 0.17619454693278938)

Высокая корреляция между уровнем безработицы и кол-вом выпустников из ВУЗов, следовательно при уменьшении кол-ва выпускников уровень безроботицы повышается. Но кол-во выпускников из учреждений СПО влияет на безработицу незначительно. Так же можем сказать, что при росте кол-ва обучающихся в целом, уровень безработицы уменьтшается